In [ ]:
# Code for ETL operations on Country-GDP data

# Importing the required libraries
import sqlite3
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

In [ ]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ['Country', 'GDP_USD_millions']
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = 'Countires_by_GDO.csv'

# Task1: Extracting information

In [ ]:
def extract(url, table_attribs):
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      df = pd.DataFrame(columns=table_attribs)
      tables = soup.find_all('tbody')
      rows = tables[2].find_all('tr')
      for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
          if col[0].find('a') is not None and '-' not in col[2]:
            data_dict = {"Country": col[0].a.string, "GDP_USD_millions": col[2].contents[0]}
            df = pd.concat([df, pd.DataFrame([data_dict])], ignore_index=True)
      return df


In [ ]:
extract(url, table_attribs)

Country GDP_USD_millions
0    United States       26,854,599
1            China       19,373,586
2            Japan        4,409,738
3          Germany        4,308,854
4            India        3,736,882
..             ...              ...
208       Anguilla                —
209       Kiribati              248
210          Nauru              151
211     Montserrat                —
212         Tuvalu               65

[213 rows x 2 columns]

# Task 2: Transform information

---

`transform` 함수는 **‘GDP_USD_millions’** 컬럼을 수정해야 합니다. 변환 과정에서 다음 사항들을 포함해야 합니다:

1️⃣ **df 데이터프레임의 'GDP_USD_millions' 컬럼의 값을 통화 형식에서 숫자 형식(부동 소수점)으로 변환하세요.**  
👉 **힌트 보기:**  
- 데이터프레임 컬럼을 리스트로 저장하세요.  
- 리스트의 내용을 반복(iterate)하면서 `split()`과 `join()` 함수를 사용해 통화 형식을 숫자 텍스트로 변환하세요.  
- 변환된 숫자 텍스트를 `float` 타입으로 형 변환하세요.

2️⃣ **변환된 값들을 모두 1000으로 나누고 소수점 둘째 자리까지 반올림하세요.**  
👉 **힌트 보기:**  
- `numpy.round()` 함수를 사용하여 값을 반올림하세요.  
- 수정된 리스트를 데이터프레임에 다시 할당하세요.

3️⃣ **컬럼 이름을 'GDP_USD_millions'에서 'GDP_USD_billions'로 변경하세요.**  
👉 **힌트 보기:**  
- `df.rename()` 함수를 사용해야 합니다.

---

In [ ]:
df = extract(url, table_attribs)

In [ ]:
def transform(df):
  df['GDP_USD_billions'] = round(df['GDP_USD_millions'].str.replace(',','').apply(pd.to_numeric, errors='coerce').fillna(0) / 1000, 2)
  df.drop(columns=['GDP_USD_millions'], inplace=True)
  return df

In [ ]:
transform(df)

Country  GDP_USD_billions
0    United States          26854.60
1            China          19373.59
2            Japan           4409.74
3          Germany           4308.85
4            India           3736.88
..             ...               ...
208       Anguilla              0.00
209       Kiribati              0.25
210          Nauru              0.15
211     Montserrat              0.00
212         Tuvalu              0.06

[213 rows x 2 columns]

# Task 3: Loading information


---

이 프로젝트의 **데이터 적재 과정은 두 단계로 나뉩니다.**

1️⃣ **변환된 데이터프레임을 CSV 파일로 저장해야 합니다.**  
이를 위해 데이터프레임 `df`와 CSV 파일 경로를 `load_to_csv()` 함수에 전달하고, 필요한 구문을 함수 내부에 추가하세요.

2️⃣ **변환된 데이터프레임을 데이터베이스의 테이블로 저장해야 합니다.**  
이를 위해 `load_to_db()` 함수에서 구현해야 하며, 이 함수는 다음 세 가지 인수를 받습니다:  
- 데이터프레임 `df`  
- SQL 데이터베이스에 대한 연결 객체 `conn`  
- 사용할 테이블 이름을 나타내는 변수 `table_name`

---

In [ ]:
def load_to_csv(df, csv_path):
  full_path = os.path.join(os.getcwd(), csv_path)
  df.to_csv(full_path, index=False)
  print(f"Dataframe was extreacted to {full_path}")

In [ ]:
def load_to_db(df, db_name, table_name):
  with sqlite3.connect(db_name) as conn:
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Dataframe was loaded to {table_name} in {db_name}")

In [ ]:
load_to_csv(df, csv_path)

Dataframe was extreacted to /content/Countires_by_GDO.csv


# Task 4: Querying the database table


---

적절한 쿼리가 시작되었고, **쿼리문(query statement)**이 **`run_query()` 함수**로 전달되었다고 가정합니다.  
이 함수는 다음과 같은 인수를 받습니다:  
- **SQL 연결 객체** `sql_connection`  
- **테이블 이름 변수** `table_name`  

`run_query()` 함수는 해당 테이블에서 쿼리문을 실행하고, 그 결과를 **필터링된 데이터프레임**으로 반환해야 합니다.  
이 데이터프레임은 단순히 출력(print)할 수 있습니다.

---



In [ ]:
def run_query(query_statement, sql_connection):
  print(query_statement)
  query_output = pd.read_sql(query_statement, sql_connection)
  print(query_output)
  return query_output

# Task 5: Logging progress

---

**로깅(logging)**은 **`log_progress()` 함수**를 사용하여 수행해야 합니다.  
이 함수는 코드 실행 중 여러 번 호출되며, **`etl_project_log.txt` 파일**에 **로그 항목(log entry)**을 추가해야 합니다.

로그 항목의 형식은 다음과 같아야 합니다:

```
<Time_stamp> : <message_text>
```

여기서:  
- **`<Time_stamp>`**: 로그가 기록된 시간 정보를 나타냅니다.  
- **`<message_text>`**: 함수에 인수로 전달되는 메시지 텍스트입니다.  

각 로그 항목은 **별도의 줄**에 기록되어야 합니다.

---


In [ ]:
from datetime import datetime

def log_progress(message):
  timestamp_format = '%Y-%h-%d %H:%M:%S'
  now = datetime.now()
  timestamp = now.strftime(timestamp_format)
  with open('etl_project_log.txt', 'a') as f:
    f.write(timestamp + ' : ' + message + '\n')

In [ ]:
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2025-01-15 04:26:16'

# Function calls


---

이제 할당된 작업을 수행하기 위해 함수 호출 순서를 설정해야 합니다.  
아래 순서에 따라 진행하세요.

---

| **작업 단계**                 | **작업 완료 시 로깅 메시지**                                |
|--------------------------------|------------------------------------------------------------|
| **기본값 선언(known values)**  | `Preliminaries complete. Initiating ETL process.`           |
| **`extract()` 함수 호출**      | `Data extraction complete. Initiating Transformation process.` |
| **`transform()` 함수 호출**    | `Data transformation complete. Initiating loading process.` |
| **`load_to_csv()` 함수 호출**  | `Data saved to CSV file.`                                  |
| **SQLite3 연결 시작**          | `SQL Connection initiated.`                                |
| **`load_to_db()` 함수 호출**   | `Data loaded to Database as table. Running the query.`      |
| **`run_query()` 함수 호출**    | `Process Complete.`                                        |
| **SQLite3 연결 닫기**          | `-`                                                        |

---

💡 **실행해야 할 쿼리문:**  
```python
f"SELECT * FROM {table_name} WHERE GDP_USD_billions >= 100"
```

---


In [ ]:
log_progress('Preliminaries complete. Initiating ETL process.')
df = extract(url, table_attribs)
log_progress('Data extraction complete. Initiating Transformation process.')
df = transform(df)
log_progress('Data transformation complete. Initiating loading process.')
load_to_csv(df, csv_path)
log_progress('Data saved to CSV file.')
log_progress('SQL Connection initiated.')
load_to_db(df, db_name, table_name)
log_progress('Data loaded to Database as table. Running the query.')
query_statement = f"SELECT * FROM {table_name} WHERE GDP_USD_billions >= 100"
with sqlite3.connect(db_name) as conn:
  run_query(query_statement, conn)
log_progress('Process Complete.')
